In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM, Bidirectional, Embedding
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from sklearn.metrics import f1_score

from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image 
import seaborn as sns
import os
import re
import glob
import cv2
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn import preprocessing
import tqdm
from numpy import loadtxt
from sklearn.utils import class_weight
from tensorflow.keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]

In [3]:
def draw_confusion_matrix(true,preds):
    conf_matx = confusion_matrix(true, preds)
    sns.heatmap(conf_matx, annot=True,annot_kws={"size": 12},fmt='g', cbar=False, cmap=plt.cm.Blues) #'viridis'
    #plt.savefig('/home/jovyan/img1.png')
    plt.show()
    
    return conf_matx

In [4]:
def plot_history(model_history, model_name):
    fig = plt.figure(figsize=(15,5), facecolor='w')
    ax = fig.add_subplot(121)
    ax.plot(model_history.history['loss'])
    ax.plot(model_history.history['val_loss'])
    ax.set(title=model_name + ': Model loss', ylabel='Loss', xlabel='Epoch')
    ax.legend(['Train', 'Val'], loc='upper left')
    ax = fig.add_subplot(122)
    ax.plot(model_history.history['accuracy'])
    ax.plot(model_history.history['val_accuracy'])
    ax.set(title=model_name + ': Model Accuracy; test='+ str(np.round(model_history.history['val_accuracy'][-1], 3)),
           ylabel='Accuracy', xlabel='Epoch')
    ax.legend(['Train', 'Val'], loc='upper left')
    #plt.savefig('/home/jovyan/img2.png')
    plt.show()
   
    
    return fig

In [5]:
def loadImages(path_data, path_labels):
    

    feat_list = []


    for filename in sorted(glob.glob(path_data), key=natural_keys): 
        feat_list.append(np.load(filename))

    x_orig = np.reshape(feat_list, (len(feat_list),32, 64))

    path = path_labels     
    labels = pd.read_csv(path, usecols=["Type", "Category"],
                       sep="," )
    y_orig = np.array(labels['Category'])

    return x_orig, y_orig

In [6]:
p_feat = '/home/jovyan/DATA_MASTER_PROJECT/LSTM/FINAL_LSTM_LEAVE_ONE_OUT/HRH_CONTROL/'

l_name_feature = []

for i in os.listdir(p_feat):
    if '.' not in i:
        l_name_feature.append(i)
    

In [7]:
l_name_feature

['cyc', 'dox', 'ket', 'olo', 'orp']

In [8]:
tot_score = []

In [9]:
for i in l_name_feature:

    p_feat = '/home/jovyan/DATA_MASTER_PROJECT/LSTM/FINAL_LSTM_LEAVE_ONE_OUT/HRH_CONTROL/{}/'.format(i)

    train_data = p_feat + 'features_train/*.npy'
    train_lab= p_feat + 'lstm_train.csv'

    val_data = p_feat + 'features_validation/*.npy'
    val_lab = p_feat + 'lstm_validation.csv'


    test_data= p_feat + 'features_test/*.npy'
    test_lab= p_feat + 'lstm_test.csv'

    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

    x_train, y_train = loadImages(train_data, train_lab)


    x_val, y_val = loadImages(val_data, val_lab)


    x_test, y_test = loadImages(test_data, test_lab)


    weights = class_weight.compute_class_weight('balanced', np.unique(y_train),y_train)


    m = Sequential()
    m.add(LSTM(64, input_shape = (x_train.shape[1],x_train.shape[2])))
    m.add(Dropout(0.2))
    m.add(Dense(1, activation='sigmoid'))
    

    opt = keras.optimizers.Adam(lr=1e-5)

    m.compile(loss= keras.losses.binary_crossentropy, optimizer=opt, metrics = ['accuracy'])


    epochs = 300

    m4_h = m.fit(x_train,y_train,

                     callbacks = [es],

                    epochs=epochs,
                    validation_data = (x_val,y_val), 

                    class_weight = weights)

    scores = m.evaluate(x_test, y_test)
    tot_score.append(scores[1]*100)

Train on 1093 samples, validate on 313 samples
Epoch 1/300
1093/1093 [==============================] - 4s 3ms/sample - loss: 0.6920 - accuracy: 0.6048 - val_loss: 0.5858 - val_accuracy: 0.7125
Epoch 2/300
1093/1093 [==============================] - 0s 235us/sample - loss: 0.6828 - accuracy: 0.6038 - val_loss: 0.5861 - val_accuracy: 0.7125
Epoch 3/300
1093/1093 [==============================] - 0s 247us/sample - loss: 0.6828 - accuracy: 0.6057 - val_loss: 0.5865 - val_accuracy: 0.7125
Epoch 4/300
1093/1093 [==============================] - 0s 216us/sample - loss: 0.6834 - accuracy: 0.6029 - val_loss: 0.5866 - val_accuracy: 0.7125
Epoch 00004: early stopping
247/247 [==============================] - 0s 121us/sample - loss: 0.4068 - accuracy: 1.0000
Train on 1106 samples, validate on 353 samples
Epoch 1/300
1106/1106 [==============================] - 2s 2ms/sample - loss: 0.6739 - accuracy: 0.5895 - val_loss: 0.6910 - val_accuracy: 0.5184
Epoch 2/300
1106/1106 [=====================

In [10]:
list(zip(tot_score, l_name_feature))

[(100.0, 'cyc'),
 (67.52577424049377, 'dox'),
 (81.5450668334961, 'ket'),
 (52.45283246040344, 'olo'),
 (63.45177888870239, 'orp')]

# For Binary Classification

In [ ]:
test_preds = m.predict(x_test)

t= []
for i in (test_preds):
    if i > 0.5:
        t.append(1)
    else:
        t.append(0)

draw_confusion_matrix(y_test, t)

In [ ]:
print(classification_report(y_test, t))